<a href="https://colab.research.google.com/github/matthewpecsok/data_engineering/blob/main/tutorials/de_streaming_kafka_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install the required kafka packages

In [15]:
!pip install kafka-python

### Import packages

In [16]:
import os
from google.colab import userdata
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
import pandas as pd


## Download and setup Kafka and Zookeeper instances

For demo purposes, the following instances are setup locally:

- Kafka (Brokers: 127.0.0.1:9092)
- Zookeeper (Node: 127.0.0.1:2181)


In [17]:
# prompt: untar the tgz file

!tar -xvzf filename.tgz

tar (child): filename.tgz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [18]:
!curl -sSOL https://downloads.apache.org/kafka/3.7.1/kafka_2.12-3.7.1.tgz
!tar -xvzf kafka_2.12-3.7.1.tgz

kafka_2.12-3.7.1/
kafka_2.12-3.7.1/LICENSE
kafka_2.12-3.7.1/NOTICE
kafka_2.12-3.7.1/bin/
kafka_2.12-3.7.1/bin/kafka-delete-records.sh
kafka_2.12-3.7.1/bin/trogdor.sh
kafka_2.12-3.7.1/bin/kafka-jmx.sh
kafka_2.12-3.7.1/bin/connect-mirror-maker.sh
kafka_2.12-3.7.1/bin/kafka-console-consumer.sh
kafka_2.12-3.7.1/bin/kafka-consumer-perf-test.sh
kafka_2.12-3.7.1/bin/kafka-log-dirs.sh
kafka_2.12-3.7.1/bin/kafka-metadata-quorum.sh
kafka_2.12-3.7.1/bin/zookeeper-server-stop.sh
kafka_2.12-3.7.1/bin/kafka-verifiable-consumer.sh
kafka_2.12-3.7.1/bin/kafka-features.sh
kafka_2.12-3.7.1/bin/kafka-acls.sh
kafka_2.12-3.7.1/bin/zookeeper-server-start.sh
kafka_2.12-3.7.1/bin/kafka-server-stop.sh
kafka_2.12-3.7.1/bin/kafka-configs.sh
kafka_2.12-3.7.1/bin/kafka-reassign-partitions.sh
kafka_2.12-3.7.1/bin/connect-plugin-path.sh
kafka_2.12-3.7.1/bin/kafka-leader-election.sh
kafka_2.12-3.7.1/bin/kafka-producer-perf-test.sh
kafka_2.12-3.7.1/bin/kafka-transactions.sh
kafka_2.12-3.7.1/bin/kafka-topics.sh
kafka_2.

Kafka with defaults

In [19]:
!ls -ltrh

total 115M
drwxr-xr-x 7 root root 4.0K Jun 18 21:32 kafka_2.12-3.7.1
drwxr-xr-x 1 root root 4.0K Oct 14 13:23 sample_data
-rw-r--r-- 1 root root 1.1K Oct 17 22:32 generator.py
-rw-r--r-- 1 root root  196 Oct 17 22:35 kafka_2.12-3.7.0.tgz
-rw-r--r-- 1 root root 115M Oct 17 22:38 kafka_2.12-3.7.1.tgz


In [20]:
!./kafka_2.12-3.7.1/bin/zookeeper-server-start.sh -daemon ./kafka_2.12-3.7.1/config/zookeeper.properties
!./kafka_2.12-3.7.1/bin/kafka-server-start.sh -daemon ./kafka_2.12-3.7.1/config/server.properties
!echo "Give the processes 10 seconds to start before proceeding."
!sleep 10

Give the processes 10 seconds to start before proceeding.


Is Kafka running?

In [21]:
!ps -ef | grep java

root        2965       1 22 22:38 ?        00:00:03 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:
root        3391       1 58 22:38 ?        00:00:06 java -Xmx1G -Xms1G -server -XX:+UseG1GC -XX:MaxG
root        3483     826  0 22:38 ?        00:00:00 /bin/bash -c ps -ef | grep java
root        3485    3483  0 22:38 ?        00:00:00 grep java


Create the kafka topics with the following specs:

- sample-streaming-data: partitions=1

In [22]:
!./kafka_2.12-3.7.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic sample-streaming-data

Created topic sample-streaming-data.


Describe the topic for details on the configuration

In [23]:
!./kafka_2.12-3.7.1/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic sample-streaming-data

Topic: sample-streaming-data	TopicId: zQ34WgYlTxe6VL58VDnmYA	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: sample-streaming-data	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


## generator python script

This script simply generates random data to publish into our topic

In [24]:
%%writefile generator.py

import sys
args = sys.argv  # a list of the arguments provided (str)
print("running generator.py", args)
iterations = int(args[1])
print(f'iterations: {iterations}')

def error_callback(exc):
    raise Exception('Error while sendig data to kafka: {0}'.format(str(exc)))

def write_to_kafka(topic_name, items):
  from kafka import KafkaProducer

  count=0
  producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])
  for message, key in items:
    producer.send(topic_name, key=key.encode('utf-8'), value=message.encode('utf-8'), partition=0).add_errback(error_callback)
    count+=1
  producer.flush()
  print("Wrote {0} messages into topic: {1}".format(count, topic_name))

import random
from time import sleep

def generate_data(rows=2):

  index_num = random.randint(0,1000000)
  print(index_num)
  keys = list([f'{index_num}'])
  msg = list([f'hello world!{index_num}'])
  data = zip(msg, keys)

  return data

for i in range(iterations):
  write_to_kafka("sample-streaming-data", generate_data())
  sleep(random.randint(0,5))



Overwriting generator.py


# write some data

In [25]:
%%script bash --bg

python generator.py 10

In [26]:
message_n = 10

from kafka import KafkaConsumer

# Kafka consumer configuration
bootstrap_servers = ['localhost:9092']  # Kafka server address
topic_name = 'sample-streaming-data'  # Kafka topic you want to read from
group_id = 'some_group'  # Consumer group ID

# Create a Kafka consumer
consumer = KafkaConsumer(
    topic_name,
    bootstrap_servers=bootstrap_servers,
    auto_offset_reset='earliest',  # Start reading at the earliest message
    enable_auto_commit=True,
    group_id=group_id,
    value_deserializer=lambda x: x.decode('utf-8')  # Assuming messages are UTF-8 encoded
)

# Read and print messages from the topic
try:
    for _ in range(message_n):
        message = next(consumer)
        print(f"Received message: {message.value}")
finally:
    # Clean up on exit
    consumer.close()


Received message: hello world!758992
Received message: hello world!210415
Received message: hello world!229729
Received message: hello world!788082
Received message: hello world!890585
Received message: hello world!148108
Received message: hello world!955064
Received message: hello world!430529
Received message: hello world!91747
Received message: hello world!171679


# UTA is no longer working with SIRI service

# UTA API


We'll retrieve realtime data from UTA and publish it to our kafka topic as a producter.

Then we'll interact with the topic as a Consumer and get the messages out of the topic

## install package needed for uta api

In [27]:
!pip install xmltodict

## create a dict of our token to save to a file for the python script

In [34]:
token_dict = {'token':userdata.get('uta')}

## dump the token data to a file

In [29]:
with open('token.json', 'w') as file:
    json.dump(token_dict, file)

In [41]:
token = userdata.get('uta')

## retrieve the data from the API and write it to the topic.

these scripts will retrieve data from the API and write the messages to the kafka topic

In [30]:
%%writefile uta_generator.py

from google.colab import userdata
import json
import xmltodict
import sys


args = sys.argv  # a list of the arguments provided (str)
print("running generator.py", args)
iterations = int(args[1])

print(f'iterations: {iterations}')





def error_callback(exc):
    raise Exception('Error while sendig data to kafka: {0}'.format(str(exc)))




# take the data retrieved from the api and write it to the kafka topic
def write_to_kafka(topic_name, items):
  from kafka import KafkaProducer

  count=0
  producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])
  print(items)

  for ref, lat, lon in items:
    print(ref, lat, lon)
    location = f'{{"Vehichle_ref":{ref},Latitude": {lat}, "Longitude": {lon}}}'
    producer.send(topic_name, value=location.encode('utf-8'), partition=0).add_errback(error_callback)
    count+=1

  producer.flush()
  print("Wrote {0} messages into topic: {1}".format(count, topic_name))


# the api call to the uta api
def get_locations(token):

    from time import sleep
    from google.colab import userdata
    import requests
    import xmltodict
    import pandas as pd
    import os

    url = f'http://api.rideuta.com/SIRI/SIRI.svc/VehicleMonitor/ByRoute?route=703&onwardcalls=true&usertoken={token}'
    print(url)
    response = requests.get(url)
    xml_dict = xmltodict.parse(response.text)
    df = pd.DataFrame(xml_dict['Siri']['VehicleMonitoringDelivery']['VehicleActivity']['MonitoredVehicleJourney'])
    print(df)
    print(df.VehicleRef.value_counts())
    print(df.shape)

    location_df = pd.json_normalize(df.VehicleLocation)
    vehicle_location = pd.merge(df['VehicleRef'],location_df,left_index=True,right_index=True)
    zip_tuple = tuple(zip(vehicle_location['VehicleRef'],vehicle_location['Latitude'], vehicle_location['Longitude']))
    sleep(6)

    return zip_tuple

# open the token file
with open('token.json', 'r') as file:
    token_dict = json.load(file)

# turn the dict value into a local var
token = token_dict['token']

# call the function multiple times to get the data
# and write it to the kafka topic

for i in range(iterations):
  write_to_kafka("sample-streaming-data", get_locations(token))



Writing uta_generator.py


## let's remind ourselves of the UTA data structure

create a function to hit the api using our token

In [37]:
def get_red_trains(token):
    from time import sleep
    from google.colab import userdata
    import requests
    import xmltodict
    import pandas as pd

    token = userdata.get('uta')
    url = f'http://api.rideuta.com/SIRI/SIRI.svc/VehicleMonitor/ByRoute?route=703&onwardcalls=true&usertoken={token}'
    response = requests.get(url)
    xml_dict = xmltodict.parse(response.text)
    df = pd.DataFrame(xml_dict['Siri']['VehicleMonitoringDelivery']['VehicleActivity']['MonitoredVehicleJourney'])
    return df


In [39]:
import requests

In [42]:
    url = f'http://api.rideuta.com/SIRI/SIRI.svc/VehicleMonitor/ByRoute?route=703&onwardcalls=true&usertoken={token}'
    requests.get(url)

<Response [503]>

hit the api and show the dataframe we have from the request.

In [38]:
df = get_red_trains(userdata.get('uta'))
df

ExpatError: not well-formed (invalid token): line 1, column 49

## get train lat/lon

conver the JSON data to a dataframe format after normalizing the lat/lon data

In [ ]:
import pandas as pd
location_df = pd.json_normalize(df.VehicleLocation)
vehicle_location = pd.merge(df['VehicleRef'],location_df,left_index=True,right_index=True)
vehicle_location

## get UTA data

this calls the script asyncronously so our cells are not blocked from execution while the API data is retrieved

In [ ]:
%%script bash --bg

python uta_generator.py 10 >> log.txt

## KAFKA Consumer for UTA

In [ ]:
message_n = 100

messages = []

from kafka import KafkaConsumer

# Kafka consumer configuration
bootstrap_servers = ['localhost:9092']  # Kafka server address
topic_name = 'sample-streaming-data'  # Kafka topic you want to read from
group_id = 'some_group'  # Consumer group ID

# Create a Kafka consumer
consumer = KafkaConsumer(
    topic_name,
    bootstrap_servers=bootstrap_servers,
    auto_offset_reset='earliest',  # Start reading at the earliest message
    enable_auto_commit=True,
    group_id=group_id,
    value_deserializer=lambda x: x.decode('utf-8')  # Assuming messages are UTF-8 encoded
)

# Read and print five messages from the topic
try:
    for _ in range(message_n):
        message = next(consumer)
        messages.append(message)
        print(f"Received message: {message.value}")
finally:
    # Clean up on exit
    consumer.close()


function for retrieve messages.

In [ ]:
message

# retrieve messages